In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
path = '~/LXPLUS/DESY_testbeam/tree_v2/'
# path = '/eos/user/s/ssuvorov/DESY_testbeam/tree_v2/'
path_loc = '~/DATA/'
file_name = path + '/g_360_200_100p'
N_iter = 4

## PRF over multiplicities

In [ ]:
PRF_function  = ROOT.TF1("PRF_function",
                         "[0]*(1+[1]*x*x + [2] * x*x*x*x) / (1+[3]*x*x+[4]*x*x*x*x)",
                         -0.027, 0.027);
PRF_function.SetLineColor(ROOT.kRed)

file = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
cl.Clear()
cl.Divide(2, 2)
for i in range(4):
    if i == 0:
        histo = file.Get('PRF_histo')
        gr = file.Get('PRF_graph')
    else:
        histo = file.Get('prf_mult/PRF_histo_' + str(i+1) + 'pad')
        gr = file.Get('prf_mult/PRF_graph_' + str(i+1) + 'pad')
    cl.cd(i+1)
#     histo.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    histo.GetYaxis().SetRangeUser(0., 1.)
    histo.Draw('colz')
#     gr.GetFunction("PRF_function").SetLineColor(ROOT.kRed)
    gr.SetLineColor(ROOT.kRed)
    gr.SetMarkerColor(ROOT.kRed)
    gr.Draw('same p')
cl.Draw()

## Charge and time PRF

In [ ]:
%jsroot on
cl.Clear()
cl.Divide(2, 2)
N_iter = 9
file = ROOT.TFile(file_name + '_iter' + str(N_iter) + '.root')
# file = ROOT.TFile(path_loc + 'phi_200_40_iter1.root')

prf_h = ROOT.TH2F('prf_h','',  180, -0.027, 0.027, 100, 0., 1.)
prf_t = ROOT.TH2F('prf_t', '', 180, -0.027, 0.027, 100, -20., 80.)
prf_tc = ROOT.TH2F('prf_tc', '', 100, 0., 1., 100, -20., 80.)

cl.cd(1)
ROOT.gPad.SetGrid()
gr = file.Get('PRF_graph')
gr.Draw('ap')
gr.GetYaxis().SetRangeUser(0., 1.)
gr.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr.GetXaxis().SetTitle('x_{cluster} - x_{track}')

cl.cd(2)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_h', 'qfrac:dx', '')
prf_h.Draw('colz')
prf_h.GetYaxis().SetRangeUser(0., 1.)
prf_h.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
prf_h.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cl.cd(3)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t', 'time[][] - time[][0]:dx', '')
prf_t.Draw('colz')
prf_t.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_t.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cl.cd(4)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_tc', 'time[][] - time[][0]:qfrac', '')
prf_tc.Draw('colz')
prf_tc.GetYaxis().SetRangeUser(0., 1.)
prf_tc.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_tc.GetXaxis().SetTitle('Q_{pad}/Q_{cluster}')

cl.Draw()

### Time PRF

In [ ]:
c.cd()
prf_t = ROOT.TH2F('prf_t', '', 180, -0.027, 0.027, 100, -20., 80.)
file.outtree.Project('prf_t', 'time[][] - time[][0]:dx[][]', '')
prf_t.Draw('colz')
prf_t.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
prf_t.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

prf_t.Draw('colz')
c.Draw()

### Profile Time PRF

In [ ]:
def profileY(h):
    gr = ROOT.TGraphErrors()
    for i in range(1, h.GetXaxis().GetNbins()):
        temp = h.ProjectionY(f"projectionsY_bin_{i}", i, i)
        if temp.GetMaximum() < 1:
            continue
        x = h.GetXaxis().GetBinCenter(i)
        y = temp.GetBinCenter(temp.GetMaximumBin())
        gr.SetPoint(gr.GetN(), x, y)
        start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
        end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
        gr.SetPointError(gr.GetN()-1, 0., 0.5*(end-start))
    return gr

def profileX(h):
    gr = ROOT.TGraphErrors()
    for i in range(22, h.GetYaxis().GetNbins()):
        temp = h.ProjectionX(f"projectionsX_bin_{i}", i, i)
        if temp.GetMaximum() < 1:
            continue
        for j in range(temp.GetXaxis().FindBin(0.), temp.GetXaxis().GetNbins()):
            temp.SetBinContent(j, 0)
        y = h.GetYaxis().GetBinCenter(i)
        x = temp.GetBinCenter(temp.GetMaximumBin())
        gr.SetPoint(gr.GetN(), x, y)
        start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
        end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
        gr.SetPointError(gr.GetN()-1, 0.5*(end-start), 0.)
    return gr

cw.cd(1)
ROOT.gPad.SetGrid()    
gr = profileY(prf_t)
gr.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
gr.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')
# gr.Fit('pol2')
gr.Draw('ap')

cw.cd(2)
ROOT.gPad.SetGrid()    
gr2 = profileX(prf_t)
gr2.GetYaxis().SetTitle('t_{pad} - t_{leading} [bins]')
gr2.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')
gr2.Fit('pol2')
gr2.Draw('ap')

cw.Draw()

### Profile charge PRF

In [ ]:
cw.Clear()
cw.Divide(2)

def profileX(h):
    gr = ROOT.TGraphErrors()
    for i in range(0, 89):#h.GetYaxis().GetNbins()):
        temp = h.ProjectionX(f"projectionsX_bin_{i}", i, i)
        if temp.GetMaximum() < 1:
            continue
        for j in range(90, 180):
            temp.SetBinContent(j, 0)
        y = h.GetYaxis().GetBinCenter(i)
        x = temp.GetBinCenter(temp.GetMaximumBin())
        gr.SetPoint(gr.GetN(), x, y)
        start = temp.GetBinCenter(temp.FindFirstBinAbove(temp.GetMaximum()/2))
        end = temp.GetBinCenter(temp.FindLastBinAbove(temp.GetMaximum()/2))
        gr.SetPointError(gr.GetN()-1, 0.5*(end-start), 0.)
    return gr

cw.cd(1)
ROOT.gPad.SetGrid()    
gr = profileY(prf_h)
gr.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr.GetXaxis().SetTitle('x_{cluster} - x_{track}')
gr.Draw('ap')

cw.cd(2)
ROOT.gPad.SetGrid()    
gr2 = profileX(prf_h)
gr2.GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
gr2.GetXaxis().SetTitle('x_{cluster} - x_{track}')
gr2.Draw('ap')


cw.Draw()

### Invert time PRF

In [ ]:
cw.cd()
cw.Clear()
cw.Divide(2)

cw.cd(1)
prf_t1 = ROOT.TH2F('prf_t1', '', 180, -0.027, 0.027, 50, -20., 30.)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t1', 'time[][1] - time[][0]:dx[][0]', '')
prf_t1.Draw('colz')
prf_t1.GetYaxis().SetTitle('t_{2nd pad} - t_{1st pad} [bins]')
prf_t1.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cw.cd(2)
prf_t2 = ROOT.TH2F('prf_t2', '', 180, -0.027, 0.027, 100, -20., 80.)
ROOT.gPad.SetGrid()
file.outtree.Project('prf_t2', 'time[][2] - time[][0]:dx', '')
prf_t2.Draw('colz')
prf_t2.GetYaxis().SetTitle('t_{3rd pad} - t_{1st pad} [bins]')
prf_t2.GetXaxis().SetTitle('x_{pad} - x_{track} [m]')

cw.Draw()

# PRF over iterations

In [ ]:
par = []
par_col = []
for i in range(5):
    par.append(ROOT.TGraphErrors())
    par_col.append(ROOT.TGraphErrors())

for it in range(N_iter):
    file = ROOT.TFile(file_name + '_iter' + str(it) + '.root')
    func = file.Get('PRF_function') 
    for i in range(5):
        par[i].SetPoint(it, it, func.GetParameter(i))
        par[i].SetPointError(it, func.GetParError(i))

# PRF over column

In [ ]:
file = ROOT.TFile(file_name + '_iter' + str(2) + '.root')
for i in range(36):
    h = file.Get(f'prf_column/PRF_histo_col_{i}')
    h.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    for j in range(5):
        par_col[j].SetPoint(i, i, PRF_function.GetParameter(j))
        par_col[j].SetPointError(i, PRF_function.GetParError(j))

In [ ]:
par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
cl.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par[i-1].GetYaxis().SetTitle(par_name[i-1])
    par[i-1].GetXaxis().SetTitle('Iteration')
    par[i-1].Draw('ap')
    
cl.Draw()

In [ ]:
c = ROOT.TCanvas('c', '', 1600, 1200)

par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
c.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par_col[i-1].GetYaxis().SetTitle(par_name[i-1])
    par_col[i-1].GetXaxis().SetTitle('Column')
    par_col[i-1].Draw('ap')
    
c.Draw()

# PRF MM vs ERAM

In [ ]:
N_iter = 5
file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_gr = file_mm.Get('PRF_graph')
PRF_er_gr = file_er.Get('PRF_graph')
PRF_mm_hi = file_mm.Get('PRF_histo')


c.Clear()
leg = ROOT.TLegend(0.2,0.2,0.4,0.4);

PRF_er_gr.GetFunction('PRF_function').SetLineColor(ROOT.kGreen)
PRF_mm_gr.GetFunction('PRF_function').SetLineColor(ROOT.kRed)
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
PRF_er_gr.GetFunction('PRF_function').GetXaxis().SetTitle('x_{pad}-x_{track}')
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetRangeUser(0., 1.)
PRF_er_gr.GetFunction('PRF_function').Draw('l')
PRF_mm_gr.GetFunction('PRF_function').Draw('same l')
leg.AddEntry(PRF_er_gr, 'ERAM2 360V', 'l')
leg.AddEntry(PRF_mm_gr, 'MM1 380V', 'l')
c.Draw()
# leg.Draw()

In [ ]:
cl.Clear()
cl.Divide(2, 2)

file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(0) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(0) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_hi = file_mm.Get('PRF_histo')
cl.cd(1)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(3)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')


file_mm_last = ROOT.TFile(folder + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er_last = ROOT.TFile(folder + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er_last.Get('PRF_histo')
PRF_mm_hi = file_mm_last.Get('PRF_histo')

cl.cd(2)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(4)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')

cl.Draw()
